In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Row

In [2]:
spark = SparkSession.builder.getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [16]:
df = spark.range(100)\
    .withColumn('rndn', F.expr("cast(randn()*10 as integer)"))\
    .withColumn('dt', F.expr("add_months(to_date('2024-01-01'), rndn)"))\
    .withColumn('dt', F.expr("case when id in (0, 10, 75) then Null else dt end"))
df.show(10)

+---+----+----------+
| id|rndn|        dt|
+---+----+----------+
|  0| -17|      null|
|  1|  -7|2023-06-01|
|  2|  14|2025-03-01|
|  3|  -4|2023-09-01|
|  4|   2|2024-03-01|
|  5| -10|2023-03-01|
|  6|   1|2024-02-01|
|  7|   6|2024-07-01|
|  8|  -4|2023-09-01|
|  9|  -8|2023-05-01|
+---+----+----------+
only showing top 10 rows



In [17]:
dfg = df.groupBy('dt').agg(*[F.expr(f'collect_list({cl})') for cl in df.columns])
dfg.show()

+----------+--------------------+--------------------+--------------------+
|        dt|    collect_list(id)|  collect_list(rndn)|    collect_list(dt)|
+----------+--------------------+--------------------+--------------------+
|2023-01-01|        [16, 74, 83]|     [-12, -12, -12]|[2023-01-01, 2023...|
|      null|         [0, 10, 75]|         [-17, 8, 3]|                  []|
|2024-09-01|[24, 58, 88, 94, 97]|     [8, 8, 8, 8, 8]|[2024-09-01, 2024...|
|2024-10-01|[20, 36, 49, 68, 99]|     [9, 9, 9, 9, 9]|[2024-10-01, 2024...|
|2023-07-01|    [14, 27, 71, 92]|    [-6, -6, -6, -6]|[2023-07-01, 2023...|
|2023-06-01|             [1, 13]|            [-7, -7]|[2023-06-01, 2023...|
|2024-08-01|            [23, 42]|              [7, 7]|[2024-08-01, 2024...|
|2022-04-01|                [22]|               [-21]|        [2022-04-01]|
|2023-09-01|      [3, 8, 56, 87]|    [-4, -4, -4, -4]|[2023-09-01, 2023...|
|2024-07-01| [7, 46, 50, 72, 76]|     [6, 6, 6, 6, 6]|[2024-07-01, 2024...|
|2025-04-01|

In [114]:
df = df.withColumn('jsn', F.expr("struct(*)").dropFields('id'))\
       .withColumn('jsn', F.expr("to_json(jsn)")).select('id', 'jsn')

df.printSchema()
df.show(truncate=False)

root
 |-- id: long (nullable = false)
 |-- jsn: string (nullable = true)

+---+------------------------------+
|id |jsn                           |
+---+------------------------------+
|0  |{"rndn":-12}                  |
|1  |{"rndn":14,"dt":"2025-03-01"} |
|2  |{"rndn":-6,"dt":"2023-07-01"} |
|3  |{"rndn":-5,"dt":"2023-08-01"} |
|4  |{"rndn":29,"dt":"2026-06-01"} |
|5  |{"rndn":0,"dt":"2024-01-01"}  |
|6  |{"rndn":-2,"dt":"2023-11-01"} |
|7  |{"rndn":4,"dt":"2024-05-01"}  |
|8  |{"rndn":-6,"dt":"2023-07-01"} |
|9  |{"rndn":0,"dt":"2024-01-01"}  |
|10 |{"rndn":-25}                  |
|11 |{"rndn":-12,"dt":"2023-01-01"}|
|12 |{"rndn":0,"dt":"2024-01-01"}  |
|13 |{"rndn":-11,"dt":"2023-02-01"}|
|14 |{"rndn":0,"dt":"2024-01-01"}  |
|15 |{"rndn":7,"dt":"2024-08-01"}  |
|16 |{"rndn":0,"dt":"2024-01-01"}  |
|17 |{"rndn":1,"dt":"2024-02-01"}  |
|18 |{"rndn":-5,"dt":"2023-08-01"} |
|19 |{"rndn":6,"dt":"2024-07-01"}  |
+---+------------------------------+
only showing top 20 rows



In [127]:
spark.stop()